Note book này được sao chép và chỉnh sửa lại từ project của [Nick Kuzmenkov ](https://www.kaggle.com/nickuzmenkov)

Notebook này tập trung vào việc xử lý dữ liệu, bao gồm các công việc
1. Loại bỏ các ảnh bị trùng lặp.
2. Định dạng lại nhãn.
3. Tạo các fold để áp dụng phương pháp Cross Validation.
4. Tạo pre-augmentation cho dữ liệu ảnh.
5. Đưa kết quả vào định dạng file Tfrecord.
### Các notebook khác của nhóm:

[Revealing Duplicates notebook](https://www.kaggle.com/nvlinhh/int3414-22-n11-revealing-duplicate)
 
[Tranning notebook](https://www.kaggle.com/congnguyen8201/int3414-22-n11-training)
 
[Submission notebook](https://www.kaggle.com/congnguyen8201/int3414-22-n11-submission)

### Imports

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import StratifiedKFold
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
import albumentations
import pandas as pd
import numpy as np
import shutil
import os

### Configuration


In [ ]:
plt.style.use('fivethirtyeight')
print(f'Using TensorFlow {tf.__version__}')

class CFG:
    
    '''
    keep these
    '''
    root = '../input/plant-pathology-2021-fgvc8/train_images'
    classes = [
        'complex', 
        'frog_eye_leaf_spot', 
        'powdery_mildew', 
        'rust', 
        'scab',
        'healthy']
    strategy = tf.distribute.get_strategy()#phân phối việc huấn luyện của Tensorflow
    batch_size = 16
    
    '''
    tune these
    '''
    img_size = 600 # kích thước ảnh
    folds = 5 # số lượng KFold n_splits
    seed = 42 # random seed (chỉ dành cho KFold)
    subfolds = 16 # số file tfrec trong mỗi fold
    transform = True # có áp dụng pre-augmentation
    epochs = 5 # (>=5) số lần áp dụng pre-augmetaion khi transform = True


## 1. Removing duplicates
Việc xác định sự trùng lặp của các ảnh được thực hiên ở **[other notebook](https://www.kaggle.com/nickuzmenkov/pp2021-duplicates-revealing)** và nhận được 50 ảnh trừn lặp. Dưới đây là cách nhóm xử lỷ các ảnh bị trùng:
1. Chỉ giữ lại một nếu các ảnh giống nhau có chung nhãn.
2. Xóa tất cả các ảnh nếu các ảnh giống nhau khác nhãn.

In [ ]:
df = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv', index_col='image')
init_len = len(df)

#đọc file
with open('../input/int3414-22-n11-revealing-duplicate/duplicates.csv', 'r') as file:
    duplicates = [x.strip().split(',') for x in file.readlines()]
print(len(duplicates))

for row in duplicates:
    unique_labels = df.loc[row].drop_duplicates().values
    if len(unique_labels) == 1:
        df = df.drop(row[1:], axis=0)
    else:
        df = df.drop(row, axis=0)
        
print(f'Dropping {init_len - len(df)} duplicate samples.')

In [ ]:
df['labels'].value_counts()

## 2. Label formatting
The initial format of space-separated string labels is inapplicable for model training. Here we change the format via `MultiLabelBinarizer` instance

In [ ]:
original_labels = df['labels'].values.copy()
original_labels

In [ ]:
#Chuyển label từ dạng string sang list 
df['labels'] = [x.split(' ') for x in df['labels']]
print(df['labels'])

#tạo onehot vector
labels = MultiLabelBinarizer(classes=CFG.classes).fit_transform(df['labels'].values)
df = pd.DataFrame(columns=CFG.classes, data=labels, index=df.index)

df.to_csv('train.csv')
display(df.head())


## 3. Making stratified folds
Để tránh imbalanced class distribution ở các fold nên nhóm sử dụng StratifiedKfolds của thư viện SKlearn. 


In [ ]:
kfold = StratifiedKFold(n_splits=CFG.folds, shuffle=True, random_state=CFG.seed)
fold = np.zeros((len(df),))

#Kfold sau khi trộn tập data, fold được tạo ra để đánh dấu từng phần tử của df thuộc tập test trong fold nào.
for i, (train_index, val_index) in enumerate(kfold.split(df.index, original_labels)):#đếm các phần tử trong tập val thuộc fold nào.
    fold[val_index] = i
    
#đêm số lần các labels suất hiện ở từng fold(giá trị 1)
value_counts = lambda x: pd.Series.value_counts(x, normalize=True)
df_occurence = pd.DataFrame({
    'origin': df.apply(value_counts).loc[1],
    'fold_0': df[fold == 0].apply(value_counts).loc[1],
    'fold_1': df[fold == 1].apply(value_counts).loc[1],
    'fold_2': df[fold == 2].apply(value_counts).loc[1],
    'fold_3': df[fold == 3].apply(value_counts).loc[1],
    'fold_4': df[fold == 4].apply(value_counts).loc[1]})
print(df_occurence)

bar = df_occurence.plot.barh(figsize=[15, 5], colormap='plasma')

folds = pd.DataFrame({
    'image': df.index,
    'fold': fold})

folds.to_csv('folds.csv', index=False)

Đối với mảng fold, giá trị j ở vị trí i thể hiện cho việc phần tử ở vị trí i thuộc fold j.
Sau khi thực hiện việc chia thành các fold, một nhãn có tỉ lệ gần bằng nhau ở các fold. 
## 4. Data pre-augmentation
Đối với việc tạo pre-augmentation cho tập dữ liệu, nhóm chọn thư viện albumentations.

In [ ]:
if CFG.transform:
    transform = albumentations.Compose([
        #ngẫu nhiên áp dụng các phương pháp dưới đây với tham số xác xuất là p.
       albumentations.RandomResizedCrop(CFG.img_size, CFG.img_size, scale=(0.9, 1), p=1), #cắt ảnh và resize
       albumentations.HorizontalFlip(p=0.5),#lật ngang
       albumentations.VerticalFlip(p=0.5),#lật dọc
       albumentations.ShiftScaleRotate(p=0.5),#ngẫu nhiên áp dụng affine transform
       albumentations.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=10, val_shift_limit=10, p=0.7),#ngẫu nhiên thay đổi các giá trị trong không gian màu HSB(hue, saturation, brightness)
       albumentations.RandomBrightnessContrast(brightness_limit=(-0.2,0.2), contrast_limit=(-0.2, 0.2), p=0.7),#Thay đổi độ tương phản
       albumentations.CLAHE(clip_limit=(1,4), p=0.5),
       albumentations.OneOf([
           albumentations.OpticalDistortion(distort_limit=1.0),
           albumentations.GridDistortion(num_steps=5, distort_limit=1.),
           albumentations.ElasticTransform(alpha=3),
       ], p=0.2),
        #thêm các noise vào ảnh
       albumentations.OneOf([
           albumentations.GaussNoise(var_limit=[10, 50]),
           albumentations.GaussianBlur(),
           albumentations.MotionBlur(),
           albumentations.MedianBlur(),
       ], p=0.2),
      albumentations.Resize(CFG.img_size, CFG.img_size),
      albumentations.OneOf([
          albumentations.JpegCompression(),
          albumentations.Downscale(scale_min=0.1, scale_max=0.15),
      ], p=0.2),
      albumentations.IAAPiecewiseAffine(p=0.2),
      albumentations.IAASharpen(p=0.2),
        #tạo ra những ô vuông nhỏ trong ảnh để ngăn overfitting  
      albumentations.Cutout(max_h_size=int(CFG.img_size * 0.1), max_w_size=int(CFG.img_size * 0.1), num_holes=5, p=0.5),
    ])
else:
    transform = None

Ví dụ về `albumentations`

In [ ]:
figure, axes = plt.subplots(5, 5, figsize=[15, 15])
axes = axes.reshape(-1,)

if transform is None:
    for i in range(len(axes)):
        image = tf.io.read_file(os.path.join(CFG.root, df.index[i]))
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, [CFG.img_size, CFG.img_size])
        image = tf.cast(image, tf.uint8)
        
        axes[i].imshow(image.numpy())
        axes[i].axis('off')

else:
    image = tf.io.read_file(os.path.join(CFG.root, df.index[CFG.seed]))
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [CFG.img_size, CFG.img_size])
    image = tf.cast(image, tf.uint8)

    for i in range(len(axes)):
        axes[i].imshow(transform(image=image.numpy())['image'])
        axes[i].axis('off')

plt.show()

## 5. Making TFRecords
### Helper functions (serialization)

In [ ]:
#decode ảnh sang ma trận, chuyển size và cast sang dạng unit8, tạo augmentation
def _serialize_image(path, transform=None):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [CFG.img_size, CFG.img_size])
    image = tf.cast(image, tf.uint8)
    
    if transform is not None:
        #tạo Augmentation cho ảnh
        image = transform(image=image.numpy())['image']
        
    return tf.image.encode_jpeg(image).numpy()


def _serialize_sample(image, image_name, label):
    feature = {
        'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])),
        'image_name': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_name])),
        'complex': tf.train.Feature(int64_list=tf.train.Int64List(value=[label[0]])),
        'frog_eye_leaf_spot': tf.train.Feature(int64_list=tf.train.Int64List(value=[label[1]])),
        'powdery_mildew': tf.train.Feature(int64_list=tf.train.Int64List(value=[label[2]])),
        'rust': tf.train.Feature(int64_list=tf.train.Int64List(value=[label[3]])),
        'scab': tf.train.Feature(int64_list=tf.train.Int64List(value=[label[4]])),
        'healthy': tf.train.Feature(int64_list=tf.train.Int64List(value=[label[5]]))}
    sample = tf.train.Example(features=tf.train.Features(feature=feature))#định dạng lưu trữ tfrec, trong đó có các feature
    return sample.SerializeToString()


def serialize_fold(fold, name, transform=None, bar=None):
    samples = []
    #Duyệt các ảnh trong fold
    for image_name, labels in fold.iterrows():
        path = os.path.join(CFG.root, image_name)#lấy đường dẫn ảnh
        image = _serialize_image(path, transform=transform)
        samples.append(_serialize_sample(image, image_name.encode(), labels))
    
    #viết vào file
    with tf.io.TFRecordWriter(name + '.tfrec') as writer:
        [writer.write(x) for x in samples]
        
    if bar is not None:
        bar.update(1)

In [ ]:
total = CFG.folds * CFG.subfolds if transform is None else CFG.folds * CFG.subfolds * CFG.epochs

with tqdm(total=total) as bar:

    for i in range(CFG.folds):

        df_fold = df[fold == i]
        #print(df_fold)
        
        folder = f'fold_{i}'
        
        #tạo folder cho các fold
        try:
            os.mkdir(folder)
        except FileExistsError:
            shutil.rmtree(folder)
            os.mkdir(folder)
        
        #chạy 80 lần khi ko tạo augmentation
        if transform is None:
            for k, subfold in enumerate(np.array_split(df_fold, CFG.subfolds)):
                name=os.path.join(folder, '%.2i-%.3i' % (k, len(subfold)))
                serialize_fold(subfold, name=name, bar=bar)
                
        #chạy total (400) lần do x5 lần tạo augmentation 
        else:
            for j in range(CFG.epochs):
                for k, subfold in enumerate(np.array_split(df_fold, CFG.subfolds)):
                    name=os.path.join(folder, '%.2i-%.3i' % (j * CFG.subfolds + k, len(subfold)))
                    serialize_fold(subfold, name=name, transform=transform, bar=bar)
#                     print(k,len(subfolds))
#                     print(name)